In [1]:
from __future__ import print_function, division
import os
import time
import torch
import pandas as pd
import numpy as np
# For showing and formatting images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# For importing datasets into pytorch
import torchvision.datasets as dataset

# Used for dataloaders
from torch.utils.data import DataLoader

# For pretrained resnet34 model
import torchvision.models as models

# For optimisation function
import torch.nn as nn
import torch.optim as optim

# For turning data into tensors
import torchvision.transforms as transforms

# For loss function
import torch.nn.functional as F

# Tensor to wrap data in
from torch.autograd import Variable

In [2]:
PATH = '/home/cell/data/plant_seedlings/model/'
!ls {PATH+"train"}

Black-grass  Common Chickweed  Loose Silky-bent   Shepherds Purse
Charlock     Common wheat      Maize		  Small-flowered Cranesbill
Cleavers     Fat Hen	       Scentless Mayweed  Sugar beet


In [3]:
batch_size = 16
sz = 224

In [4]:
## Image loaders
## Dataset transforms puts the images in tensor form
normalise = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_raw = dataset.ImageFolder(PATH+"train", transform=transforms.Compose([transforms.RandomResizedCrop(sz),
                                                                            transforms.RandomHorizontalFlip(),
                                                                            transforms.ToTensor(),
                                                                           normalise]))
train_loader = DataLoader(train_raw, batch_size=batch_size, shuffle=True, num_workers=4)

valid_raw = dataset.ImageFolder(PATH+"valid", transform=transforms.Compose([transforms.CenterCrop(sz),
                                                                            transforms.ToTensor(),
                                                                           normalise]))
valid_loader = DataLoader(valid_raw, batch_size=batch_size, shuffle=False, num_workers=4)

In [5]:
#print(train_raw.classes)
#print(train_raw.class_to_idx)

In [6]:
## Create resnet model
resnet34=models.resnet34(pretrained=True)

num_ftrs = resnet34.fc.in_features

## Freeze all but the last layers
for param in resnet34.parameters():
    ## Each tensor has the flag requires_grad, setting it to false allows freezes
    ## the parmaeter associated with it
    param.requires_grad = False
    
# Parameters of newly constructed modules have requires_grad=True by default
## Create new modules that will become final layer
num_ftrs = resnet34.fc.in_features
print(num_ftrs)
## Give final layers a linear transform with twelve outputs one for each category
resnet34.fc = nn.Linear(num_ftrs, 13)

## Create new model and tell it whether the computer has a GPU or not

## Loss function and optimiser
criterion = nn.CrossEntropyLoss().cuda()
optimiser = optim.Adam(resnet34.fc.parameters(), lr=0.001, weight_decay=0.001)

512


In [7]:
def train(epochs):
    #epoch=1
    resnet34.train()
    time_secs = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        start_time = time.time()
        #print(batch_idx)
        data, target = Variable(data), Variable(target)
        optimiser.zero_grad()
        output = resnet34(data)
        #print("Output: ", output)
        #print("Target: ", target)
        loss=criterion(output, target)
        loss.backward()
        optimiser.step()
        time_secs += (time.time() - start_time)
        if batch_idx % 10 == 0:
            print("Run time for 10 batches was: ", time_secs)
            print('Train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx*len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data[0]))
            time_secs = 0
            #break;

In [8]:
def validation():
    resnet34.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:
        data, target = Variable(data, volatile = True), Variable(target)
        output=resnet34(data)
        #print("output: ", output)
        #print("target: ", target)
        test_loss += criterion(output, target).data[0]
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    test_loss /= len(valid_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(valid_loader.dataset),
    100. * correct / len(valid_loader.dataset)))

In [9]:
## Loop through epochs training data and then testing it
for epoch in range(1,10):
    train(epoch)
    validation()

Run time for 10 batches was:  3.4495320320129395
Train epoch: 1 [0/3806 (0%)]	Loss: 2.774097
Run time for 10 batches was:  26.35176110267639
Train epoch: 1 [160/3806 (4%)]	Loss: 2.913017
Run time for 10 batches was:  26.201773166656494
Train epoch: 1 [320/3806 (8%)]	Loss: 2.460073
Run time for 10 batches was:  26.396099090576172
Train epoch: 1 [480/3806 (13%)]	Loss: 2.263209
Run time for 10 batches was:  26.5547878742218
Train epoch: 1 [640/3806 (17%)]	Loss: 1.993510
Run time for 10 batches was:  25.955686807632446
Train epoch: 1 [800/3806 (21%)]	Loss: 2.410173
Run time for 10 batches was:  26.032048225402832
Train epoch: 1 [960/3806 (25%)]	Loss: 2.123491
Run time for 10 batches was:  26.232600927352905
Train epoch: 1 [1120/3806 (29%)]	Loss: 1.959553
Run time for 10 batches was:  26.14695405960083
Train epoch: 1 [1280/3806 (34%)]	Loss: 2.104743
Run time for 10 batches was:  26.224750995635986
Train epoch: 1 [1440/3806 (38%)]	Loss: 1.782087
Run time for 10 batches was:  26.4368345737457

Run time for 10 batches was:  25.771840572357178
Train epoch: 4 [1920/3806 (50%)]	Loss: 1.063787
Run time for 10 batches was:  26.04831027984619
Train epoch: 4 [2080/3806 (55%)]	Loss: 1.201853
Run time for 10 batches was:  25.639350652694702
Train epoch: 4 [2240/3806 (59%)]	Loss: 1.365049
Run time for 10 batches was:  25.76971125602722
Train epoch: 4 [2400/3806 (63%)]	Loss: 0.682599
Run time for 10 batches was:  25.950114965438843
Train epoch: 4 [2560/3806 (67%)]	Loss: 1.086460
Run time for 10 batches was:  25.744615077972412
Train epoch: 4 [2720/3806 (71%)]	Loss: 0.895966
Run time for 10 batches was:  25.754014253616333
Train epoch: 4 [2880/3806 (76%)]	Loss: 1.333724
Run time for 10 batches was:  25.76490068435669
Train epoch: 4 [3040/3806 (80%)]	Loss: 1.164353
Run time for 10 batches was:  26.095804691314697
Train epoch: 4 [3200/3806 (84%)]	Loss: 1.468507
Run time for 10 batches was:  26.097576379776
Train epoch: 4 [3360/3806 (88%)]	Loss: 1.068233
Run time for 10 batches was:  25.877


Test set: Average loss: 0.0942, Accuracy: 451/944 (48%)

Run time for 10 batches was:  2.8943941593170166
Train epoch: 8 [0/3806 (0%)]	Loss: 1.073682
Run time for 10 batches was:  25.732027053833008
Train epoch: 8 [160/3806 (4%)]	Loss: 0.996754
Run time for 10 batches was:  25.954986095428467
Train epoch: 8 [320/3806 (8%)]	Loss: 0.954607
Run time for 10 batches was:  25.840912580490112
Train epoch: 8 [480/3806 (13%)]	Loss: 1.496408
Run time for 10 batches was:  26.149842023849487
Train epoch: 8 [640/3806 (17%)]	Loss: 1.090403
Run time for 10 batches was:  25.899043798446655
Train epoch: 8 [800/3806 (21%)]	Loss: 0.940197
Run time for 10 batches was:  25.7549090385437
Train epoch: 8 [960/3806 (25%)]	Loss: 1.046291
Run time for 10 batches was:  26.162492275238037
Train epoch: 8 [1120/3806 (29%)]	Loss: 1.261951
Run time for 10 batches was:  25.876591205596924
Train epoch: 8 [1280/3806 (34%)]	Loss: 1.148438
Run time for 10 batches was:  25.98087239265442
Train epoch: 8 [1440/3806 (38%)]	Lo